In [ ]:
import cx_Oracle
import pandas as pd
from datetime import datetime

In [ ]:
# Load CSV
df = pd.read_csv("scraped data.csv")

In [ ]:

# Connect to Oracle
conn = cx_Oracle.connect("bank_reviews", "BankPass123", "localhost:1521/XEPDB1")
cursor = conn.cursor()

In [ ]:
# Dictionary to map bank name to bank_id
bank_ids = {}

In [ ]:
# Insert banks (avoid duplicates)
unique_banks = df['bank'].unique()
for bank_name in unique_banks:
    cursor.execute("SELECT bank_id FROM banks WHERE name = :1", [bank_name])
    result = cursor.fetchone()
    if result:
        bank_ids[bank_name] = result[0]
    else:
        cursor.execute("""
            INSERT INTO banks (name) VALUES (:1)
        """, [bank_name])
        conn.commit()
        cursor.execute("SELECT bank_id FROM banks WHERE name = :1", [bank_name])
        bank_ids[bank_name] = cursor.fetchone()[0]


In [ ]:
# Insert reviews
for _, row in df.iterrows():
    try:
        review_text = str(row['review'])[:4000]  # truncate if too long
        rating = int(row['rating']) if not pd.isna(row['rating']) else None
        review_date = pd.to_datetime(row['date'], errors='coerce')
        if pd.isna(review_date):
            review_date = datetime.today()
        bank_name = row['bank']
        bank_id = bank_ids.get(bank_name)

        cursor.execute("""
            INSERT INTO reviews (bank_id, reviewer_name, review_text, rating, review_date)
            VALUES (:1, :2, :3, :4, :5)
        """, (bank_id, "Anonymous", review_text, rating, review_date))
    except Exception as e:
        print(f"Error inserting row: {e}")


In [ ]:
# Finish
conn.commit()
cursor.close()
conn.close()

In [ ]:
print("All data inserted successfully.")